In [3]:
# Import dependencies and libraries
from pymongo import MongoClient
import json
import requests
from pprint import pprint
import pandas as pd
from datetime import datetime

In [4]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

# Assign the database to a variable name
db = mongo['crimes']

# Assign the collection to a variable
chicago = db['chicago']

In [5]:
# Collect the data from the API
url = "https://data.cityofchicago.org/resource/ijzp-q8t2.json?$$app_token=7NHu8Y4PkgGZrMzrNtJwJ20bm"
response = requests.get(url)
list_of_crimes = response.json()

In [6]:
# Select an item from the returned data to view
item_to_view = list_of_crimes[0]
pprint(item_to_view)

{':@computed_region_43wa_7qmu': '46',
 ':@computed_region_6mkv_f3dw': '14917',
 ':@computed_region_awaf_s7ux': '48',
 ':@computed_region_bdys_3d7i': '91',
 ':@computed_region_d3ds_rm58': '136',
 ':@computed_region_d9mm_jgwp': '15',
 ':@computed_region_rpca_8um6': '44',
 ':@computed_region_vrxf_vc4k': '29',
 'arrest': False,
 'beat': '1231',
 'block': '003XX S LOOMIS ST',
 'case_number': 'JG400783',
 'community_area': '28',
 'date': '2023-08-28T00:00:00.000',
 'description': 'TO VEHICLE',
 'district': '012',
 'domestic': False,
 'fbi_code': '14',
 'id': '13190952',
 'iucr': '1320',
 'latitude': '41.877274235',
 'location': {'human_address': '{"address": "", "city": "", "state": "", '
                               '"zip": ""}',
              'latitude': '41.877274235',
              'longitude': '-87.661873779'},
 'location_description': 'STREET',
 'longitude': '-87.661873779',
 'primary_type': 'CRIMINAL DAMAGE',
 'updated_on': '2023-09-04T15:42:34.000',
 'ward': '27',
 'x_coordinate': 

In [7]:
# Remove all documents currently in the chicago collection. 
#This is a precautionary measure for times where retrieving data happens at multiple, inconsistent times. 
#It allows us to start fresh.
chicago.delete_many({})

In [8]:
# Loop through the list of crimes and only add those that are not currently in the collection.
for crime_to_add in list_of_crimes:
    if chicago.find_one({"id":crime_to_add["id"]}) == None:
        chicago.insert_one(crime_to_add)
    else: print(f"Crime is already in the database collection.")


In [9]:
# Filter out any data that doesn't contain a community_area.
query = {"community_area": None}
chicago.delete_many(query)

In [10]:
# Filter out any data that doesn't contain a latitude.
query = {"latitude": None}
chicago.delete_many(query)

In [11]:
# Filter out any data that doesn't contain a longitude.
query = {"longitude": None}
chicago.delete_many(query)

In [12]:
# Change the date formatting. 
for document in chicago.find():
    datetime_string = document["date"]
    python_datetime = datetime.strptime(datetime_string, "%Y-%m-%dT%H:%M:%S.%f")
    date = python_datetime.date().strftime("%m/%d/%Y")
    time = python_datetime.time().strftime("%H:%M")
    chicago.update_one(
        {"id": document["id"]},
        {
            "$set":{
                "occurance_date": date,
                "occurance_time": time
            }
        }
    )

In [13]:
# The data may come through with mutliple dates. Filter out all days, except what is predicted to be the latest full day.
available_dates = chicago.find().sort("occurance_date", -1).distinct("occurance_date")
if len(available_dates)>2:
    selected_date = available_dates[len(available_dates)-2]
else:
    selected_date = available_dates[len(available_dates)-1]

query = {"occurance_date": {"$ne": selected_date}}
chicago.delete_many(query)

In [14]:
# Group by community_area with subgroups of primary_types. Save results as a new collection called chicago_community_by_crime.
pipeline = [
    {
        "$group":{
            "_id":{
                "community_area":"$community_area",
                "primary_type":"$primary_type"
            },
            "count":{"$sum":1}
        }
    },
    {
        "$group":{
            "_id": "$_id.community_area",
            "crime":{
                "$push":{
                    "type":"$_id.primary_type",
                    "count":"$count"
                }
            }
        }
    },
    {
        "$out":"chicago_community_by_crime"
    }
]

chicago_community_by_crime = list(chicago.aggregate(pipeline))

In [15]:
# Group by primary_types with subgroups of community_area. Save results as a new collection called chicago_crime_by_community.
pipeline = [
    {
        "$group":{
            "_id":{
                "primary_type":"$primary_type",
                "community_area":"$community_area",
            },
            "count":{"$sum":1}
        }
    },
    {
        "$group":{
            "_id": "$_id.primary_type",
            "communities":{
                "$push":{
                    "community":"$_id.community_area",
                    "count":"$count"
                }
            }
        }
    },
    {
        "$out":"chicago_crime_by_community"
    }
]

chicago_crime_by_community = list(chicago.aggregate(pipeline))

In [16]:
# Group by primary_types with subgroups of occurance_time. Save results as a new collection called chicago_crime_by_time.
pipeline = [
    {
        "$group":{
            "_id":{
                "primary_type":"$primary_type",
                "occurance_time":"$occurance_time",
            },
            "count":{"$sum":1}
        }
    },
    {
        "$group":{
            "_id": "$_id.primary_type",
            "times":{
                "$push":{
                    "time":"$_id.occurance_time",
                    "count":"$count"
                }
            },
        }
    },
    {
        "$out":"chicago_crime_by_time"
    }
]

chicago_crime_by_time = list(chicago.aggregate(pipeline))

In [15]:
# USE THIS CODE ONLY IF WE CHOOSE TO USE MULTIPLE DATES - USE IT AS A TEMPLATE FOR THE OTHER AGGREGATIONS AS WELL

# Group by crime and then time - take the average if multiple days. Save results as a new collection called chicago_by_time.
#pipeline = [
#    {
#        "$group":{
#            "_id":{
#                "type":"$primary_type",
#                "time":"$occurance_time",
#                "date":"$occurance_date",
#            },
#            "count":{"$sum":1}
#        }
#    },
#    {
#        "$group":{
#            "_id":{
#                "type":"$_id.type",
#                "time":"$_id.time"
#            },
#            "dates":{
#                "$push":{
#                    "date":"$_id.date",
#                    "count":"$count"
#                }
#            }
#        }
#    },
#    {
#        "$group":{
#            "_id": "$_id.type",
#            "occurances":{
#                "$push":{
#                    "time":"$_id.time",
#                    "dates":"$dates"
#                }
#            }
#        }
#    },
#    {
#        "$unwind":"$occurances"
#    },
#    {
#        "$unwind":"$occurances.dates"
#    },
#    {
#        "$group":{
#            "_id":{
#                "type":"$_id",
#                "time":"$occurances.time"
#            },
#            "average_count":{"$avg":"$occurances.dates.count"}
#        }
#    },
#    {
#        "$group":{
#            "_id":"$_id.type",
#            "times":{
#                "$push":{
#                    "time":"$_id.time",
#                    "count":"$average_count"
#                }
#            }
#        }
#    },
#    {
#        "$out":"chicago_by_time"
#    }
#]

#chicago_by_time = list(chicago.aggregate(pipeline))